# Анализ пользовательского поведения в мобильном приложении

Цель исследования - провести анализ поведения пользователей мобильного приложения компании, занимающейся продажей продуктов питания.   
В рамках исследования требуется изучить воронку продаж, после этого исследовать с помощью A/A/B-тестирования последствия изменения шрифта в приложении.   

#### Навигация по проекту:

<a href='#section1'>1. Предобработка данных <br></a>
<a href='#section2'>2. Анализ данных <br></a>
<a href='#section3'>3. Анализ воронки событий <br></a>
<a href='#section4'>4. Изучение результатов эксперимента <br></a>

###  <a id='section2'> Шаг 2. Предобработка данных </a>

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats as st
import numpy as np
import math as mth

In [6]:
# замена наименований для столбцов
new_names = ['event_name', 'device_id', 'event_date', 'exp_id']
products.set_axis(new_names, axis = 'columns', inplace = True)

In [7]:
# подсчет суммарного количества пропусков в таблицах
products.isnull().sum()

event_name    0
device_id     0
event_date    0
exp_id        0
dtype: int64

In [8]:
# проверка на наличие дубликатов
products.duplicated().sum()

413

In [9]:
# удаление дубликатов
products = products.drop_duplicates().reset_index(drop = True)

In [10]:
# приведение типа данных
products['event_date'] = pd.to_datetime(products['event_date'], unit = 's')

In [11]:
# добавление столбца с датой
products['event_day'] = products['event_date'].astype('datetime64[D]')

###### Вывод

Таблица products состоит из:  
- EventName — название события;  
- DeviceIDHash — уникальный идентификатор пользователя;  
- EventTimestamp — время события;  
- ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная. 

По результату предобработки данных:
- заменены наименования у столбцов;
- пропуски не обнаружены;
- найденные дубликаты удалены;
- приведен формат типа данных для столбца event_date;
- добавлен столбец event_day, содержащий информацию о дне события.

###  <a id='section2'> Шаг 2. Анализ данных </a>

In [12]:
# подсчет количества событий в логе
count_events = products['event_name'].agg({'nunique'})
display('Количество уникальных событий в логе:', count_events)

'Количество уникальных событий в логе:'

nunique    5
Name: event_name, dtype: int64

In [13]:
# подсчет общего количества событий в логе
count_events_total = products['event_name'].agg({'count'})
display('Количество событий в логе:', count_events_total)

'Количество событий в логе:'

count    243713
Name: event_name, dtype: int64

In [14]:
# подсчет количества пользователей в логе
count_users = products['device_id'].agg({'nunique'})
display('Количество уникальных пользователей в логе:', count_users)

'Количество уникальных пользователей в логе:'

nunique    7551
Name: device_id, dtype: int64

In [15]:
# подсчет среднего количества событий на одного пользователя
events_per_users = products.pivot_table(index = 'device_id', values = 'event_name', aggfunc = 'count')
events_per_users_mean = events_per_users['event_name'].mean()
print('Среднее количество событий на одного пользователя:', events_per_users_mean.round(0))


Среднее количество событий на одного пользователя: 32.0


In [16]:
# определение периода данных
print('Дата начала периода:', products['event_day'].min())
print('Дата окончания периода:', products['event_day'].max())

Дата начала периода: 2019-07-25 00:00:00
Дата окончания периода: 2019-08-07 00:00:00


In [18]:
# отбор данных по дате c 1 августа

products_total = products.query('event_day >= "2019-08-01"')
products_total['event_day'].min()

Timestamp('2019-08-01 00:00:00')

In [19]:
# подсчет количества потерянных событий и пользователей в логе
count_events_new = products_total['event_name'].agg({'count'})
count_users_new = products_total['device_id'].agg({'nunique'})

diff_events = count_events_total - count_events_new
diff_users = count_users - count_users_new
print('Количество потерянных событий:', diff_events)
print('Количество потерянных пользователей:', diff_users)

Количество потерянных событий: count    2826
Name: event_name, dtype: int64
Количество потерянных пользователей: nunique    17
Name: device_id, dtype: int64


In [20]:
# проверка на наличие пользователей из всех экспериментальных групп
exp_products = products_total.pivot_table(index='exp_id', values='device_id', aggfunc='count')
display(exp_products)

,device_id
exp_id,
246,79302
247,77022
248,84563


###### Вывод

Анализ и проверка данных позволили сделать следующие выводы:
- Общее количество событий - 243713;
- Количество уникальных событий - 5;
- Количество уникальных пользователей - 7551;
- Среднее количество событий на одного уникального пользователя - 32;
- Первоначально получены данные за период с 25 июля 2019 по 07 августа 2019 года.
- Построенная гистограмма распределения даты позволила отбросить данные начиная с 31 июля, поскольку они являются выбросами - слишком маленькие значения;
- После отброса данных потеряно 2826 событий и 17 пользователей, по сравнению с общим количеством данных это небольшие значения;
- Проведена проверка на наличие пользователей всех трех экспериментальных групп.

###  <a id='section3'> Шаг 3. Анализ воронки событий </a>

In [21]:
# анализ частоты событий
events_count = products_total['event_name'].value_counts()
events_count

MainScreenAppear           117328
OffersScreenAppear          46333
CartScreenAppear            42303
PaymentScreenSuccessful     33918
Tutorial                     1005
Name: event_name, dtype: int64

In [22]:
# анализ количества событий по пользователям
events = products_total.groupby(['event_name']).agg({'device_id': ['nunique', 'count']}).reset_index()
events.columns = ['event_name', 'unique_users', 'total_users']
events = events.sort_values(by = 'unique_users', ascending = False)
events['unique_users_new'] = events['unique_users'].shift(1)
events['weight'] = (events['unique_users'] / events['unique_users_new']).round(2)
display(events)

,event_name,unique_users,total_users,unique_users_new,weight
1,MainScreenAppear,7419,117328,NaN,NaN
2,OffersScreenAppear,4593,46333,7419.0,0.62
0,CartScreenAppear,3734,42303,4593.0,0.81
3,PaymentScreenSuccessful,3539,33918,3734.0,0.95
4,Tutorial,840,1005,3539.0,0.24


Предполагаемый порядок событий:

1. MainScreenAppear - появление на главной странице
2. OffersScreenAppear - появление на экране с предложениями
3. CartScreenAppear -  появление на экране с корзиной
4. PaymentScreenSuccessful	- успешная оплата

Tutorial - консультация. Данный вид события зачастую может не присутствовать в последовательности, не все пользователи пользуются предлагаемой консультацией.

In [23]:
# отбор данных без события Tutorial
products_full = products_total.query('event_name != "Tutorial"')
products_full_grouped = products_full.groupby(['event_name']).agg({'device_id': ['nunique', 'count']}).reset_index()
products_full_grouped.columns = ['event_name', 'unique_users', 'total_users']
products_full_grouped = products_full_grouped.sort_values(by = 'unique_users', ascending = False)
products_full_grouped['unique_users_new'] = products_full_grouped['unique_users'].shift(1)
products_full_grouped['weight'] = (products_full_grouped['unique_users'] / products_full_grouped['unique_users_new']).round(2)
display(products_full_grouped)

,event_name,unique_users,total_users,unique_users_new,weight
1,MainScreenAppear,7419,117328,NaN,NaN
2,OffersScreenAppear,4593,46333,7419.0,0.62
0,CartScreenAppear,3734,42303,4593.0,0.81
3,PaymentScreenSuccessful,3539,33918,3734.0,0.95


In [24]:
# построение воронки продаж
fig = go.Figure(go.Funnel(
    y = products_full_grouped['event_name'],
    x = products_full_grouped['unique_users'],
    ))
fig.show() 

###### Вывод

Анализ воронки продаж показал следующее:
- в таблице events_count посчитана частота событий в логе, наиболее часто встречающееся событие - посещение главной страницы

- в таблице events   
  в столбце unique_users посчитано количество уникальных пользователей, совершивших каждое событие
  в столбце weight показана доля пользователей, которые хотя бы раз совершали то или иное событие
  
- построена следующая последовательность событий: MainScreenAppear -> OffersScreenAppear -> CartScreenAppear -> PaymentScreenSuccessful 

- построена воронка продаж:  
  62% пользователей от числа тех, кто зашел на главный экран перешли на страницу с предложениями товаров  
  81,2% пользователей от числа тех, кто зашел на страницу с предложениями товаров, добавили товары в корзину  
  94,7% пользователей от числа тех, кто добавил товар в корзину, совершил в итоге покупку  
Больше всего пользователей терятся при переходе с главной страницы на страницу с товарами.  
  47,7% пользователей доходит от первого события до оплаты  

###  <a id='section4'> Шаг 4. Изучение результатов эксперимента </a>

In [25]:
# количество пользователей в каждой экспериментальной группе
# 250 - это контрольная объединенная группа 246+247
exp = products_full.groupby(['exp_id']).agg({'device_id':'nunique'}).reset_index()
exp.columns = ['exp_id', 'count']
exp.loc[3] = ['250', exp.loc[0, 'count']+exp.loc[1, 'count']]
display(exp)

,exp_id,count
0,246,2483
1,247,2512
2,248,2535
3,250,4995


In [26]:
# отбор данных по группам 246 и 247
exp_groupА = products_full.query('exp_id == 246')
exp_groupA1 = products_full.query('exp_id == 247')

In [27]:
# количество пользователей, совершивших каждое из событий для группы 246
exp_groupА_gr = exp_groupА.groupby(['event_name']).agg({'device_id': ['nunique', 'count']}).reset_index()
exp_groupА_gr.columns = ['event_name', '246', 'count_246']
exp_groupА_gr['ratio_246'] = (exp_groupА_gr['count_246'] / exp_groupА_gr['246']).round(2)
exp_groupА_gr = exp_groupА_gr.sort_values(by = '246', ascending = False)
display(exp_groupА_gr)

,event_name,246,count_246,ratio_246
1,MainScreenAppear,2450,37676,15.38
2,OffersScreenAppear,1542,14767,9.58
0,CartScreenAppear,1266,14690,11.60
3,PaymentScreenSuccessful,1200,11852,9.88


In [28]:
# количество пользователей, совершивших каждое из событий для группы 247
exp_groupА1_gr = exp_groupA1.groupby(['event_name']).agg({'device_id': ['nunique', 'count']}).reset_index()
exp_groupА1_gr.columns = ['event_name', '247', 'count_247']
exp_groupА1_gr['ratio_247'] = (exp_groupА1_gr['count_247'] / exp_groupА1_gr['247']).round(2)
exp_groupА1_gr = exp_groupА1_gr.sort_values(by = '247', ascending = False)
display(exp_groupА1_gr)

,event_name,247,count_247,ratio_247
1,MainScreenAppear,2476,39090,15.79
2,OffersScreenAppear,1520,15179,9.99
0,CartScreenAppear,1238,12434,10.04
3,PaymentScreenSuccessful,1158,9981,8.62


In [29]:
# отбор данных по группе 248
exp_groupB = products_full.query('exp_id == 248')

In [30]:
# количество пользователей, совершивших каждое из событий для группы 248
exp_groupB_gr = exp_groupB.groupby(['event_name']).agg({'device_id': ['nunique', 'count']}).reset_index()
exp_groupB_gr.columns = ['event_name', '248', 'count_248']
exp_groupB_gr['ratio_248'] = (exp_groupB_gr['count_248'] / exp_groupB_gr['248']).round(2)
exp_groupB_gr = exp_groupB_gr.sort_values(by = '248', ascending = False)
display(exp_groupB_gr)

,event_name,248,count_248,ratio_248
1,MainScreenAppear,2493,40562,16.27
2,OffersScreenAppear,1531,16387,10.70
0,CartScreenAppear,1230,15179,12.34
3,PaymentScreenSuccessful,1181,12085,10.23


In [31]:
# отбор данных по объединенной контрольной группе 250 = 246 + 247
exp_groupА_total = products_full.query('exp_id != 248')

In [32]:
# количество пользователей, совершивших каждое из событий для объединенной контрольной группы - 250
exp_groupА_total_gr = exp_groupА_total.groupby(['event_name']).agg({'device_id': ['nunique', 'count']}).reset_index()
exp_groupА_total_gr.columns = ['event_name', '250', 'count_250']
exp_groupА_total_gr['ratio_250'] = (exp_groupА_total_gr['count_250'] / exp_groupА_total_gr['250']).round(2)
exp_groupА_total_gr = exp_groupА_total_gr.sort_values(by = '250', ascending = False)
display(exp_groupА_total_gr)

,event_name,250,count_250,ratio_250
1,MainScreenAppear,4926,76766,15.58
2,OffersScreenAppear,3062,29946,9.78
0,CartScreenAppear,2504,27124,10.83
3,PaymentScreenSuccessful,2358,21833,9.26


In [33]:
# построение общей таблицы с данными 

groups_total = exp_groupА_gr.merge(exp_groupА1_gr, on ='event_name', how ='left')\
                            .merge(exp_groupB_gr, on ='event_name', how ='left')\
                            .merge(exp_groupА_total_gr, on ='event_name', how ='left')


df_total = groups_total.loc[:, ['event_name', '246', '247', '248', '250']]

# добавление данных для trials отдельной строкой - total
df_total.loc[4] = ['total', exp.loc[0, 'count'], exp.loc[1, 'count'], exp.loc[2, 'count'], exp.loc[3, 'count']]


display(df_total)

,event_name,246,247,248,250
0,MainScreenAppear,2450,2476,2493,4926
1,OffersScreenAppear,1542,1520,1531,3062
2,CartScreenAppear,1266,1238,1230,2504
3,PaymentScreenSuccessful,1200,1158,1181,2358
4,total,2483,2512,2535,4995


In [34]:
# общая функция для проведения всех тестов

def test(suc_1, suc_2, trial_1, trial_2):
    successes = np.array([suc_1, suc_2])
    trials = np.array([trial_1, trial_2])
    
    alpha = .05 # критический уровень статистической значимости
    # пропорция успехов в первой группе:
    p1 = successes[0]/trials[0]
    # пропорция успехов во второй группе:
    p2 = successes[1]/trials[1]
    # пропорция успехов в комбинированном датасете:
    p_combined = (successes[0] + successes[1]) / (trials[0] + trials[1])

    # разница пропорций в датасетах
    difference = p1 - p2 

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/trials[0] + 1/trials[1]))

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)  

    # считаем статистику в ст.отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(
        p_combined * (1 - p_combined) * (1 / trials[0] + 1 / trials[1])
    )

    # задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
    distr = st.norm(0, 1)
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    print('p-значение: ', p_value)
    
    alpha_cor = alpha / 16
    if p_value < alpha_cor:
        print('Отвергаем нулевую гипотезу: между долями есть значимая разница')
    else:
        print('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными') 

Для перехода к A/B тестированию необходимо провести A/A тест, чтобы убедиться, что:
- на результаты не влияют аномалии и выбросы в генеральной совокупности;
- инструмент «деления» трафика работает безошибочно;
- данные отправляются в системы аналитики корректно.

Гипотеза для проведения A/A теста - различий в конверсии между сравниваемыми контрольными группами А и А, которые пользовались сайтом со старым штрифтом, нет.

Если две контрольные группы окажутся равны, можно быть уверенными в точности проведенного тестирования. Если же между значениями A и A будут существенные различия, это поможет обнаружить факторы, которые привели к искажению результатов. Сравнение контрольных групп также помогает понять, сколько времени и данных потребуется для дальнейших тестов.

In [35]:
# Проверка на наличие разницы между 246 и 247
# аналогичных образом выполнена проверка между 247 и 248, между 250 и 248, между 246 и 248

for i in range(0,4):
    test(df_total.loc[i, '246'], df_total.loc[i, '247'], df_total.loc[4, '246'], df_total.loc[4, '247'])

p-значение:  0.7526703436483038
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.24786096925282264
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.22867643757335676
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.11446627829276612
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными


#### A/B тестирование

Гипотеза для проведения A/B теста - различий в конверсии между сравниваемыми контрольными группами А и B, которые пользовались сайтом со старым штрифтом и с новым, нет.

###### Вывод

Тестирование A/A  
Для всех событий разница в пропорциях при указанных размерах выборок недостаточна, чтобы говорить о наличии статистически значимого различия. Тест A/A успешен, разбиение на группы работает корректно.  

Тестирование A/B  
Первоначально для всех гипотез был взят уровень значимости = 0,05. Всего было проведено 16 тестов, после 5 теста результат стал один и тот же - отвергаем нулевую гипотезу.   
Проведена корректировка по методу Бонферрони уровня значимости, итоговый уровень значимости alpha_cor был посчитан с учетом количества проводимых тестов. Итоговый результат A/B тестирования показал, что разницы в выборках нет, это значит, что изменение шрифта не влияет на конверсию.
